# TODO 

- Special keys treatment 
- user identification 
- improve logger (possibility to name the csv and to stop the app without restarting the kernel)
- 5 samples for each profile 
- Similarity measures (std, classification, nearest neighbor)

In [51]:
import pandas as pd 
import numpy as np
from datetime import datetime
from os import listdir
from os.path import isfile, join
import os

# Data import from keylogger

In [52]:
file_path = 'keylog.txt'
columns = ['Timestamp', 'Key', 'Action']
columns_bis = ['Timestamp', 'Key', 'Action', 'Shift']
dataframe = pd.read_csv(file_path, sep=";", names=columns, encoding = "ISO-8859-1")

display(dataframe)


,Timestamp,Key,Action
0,"2021-10-20 19:58:48,582",Key.shift,press
1,"2021-10-20 19:58:48,861",'B',press
2,"2021-10-20 19:58:48,982",'B',release
3,"2021-10-20 19:58:49,101",Key.shift,release
4,"2021-10-20 19:58:49,700",'o',press
5,"2021-10-20 19:58:49,830",'o',release
6,"2021-10-20 19:58:49,987",'n',press
7,"2021-10-20 19:58:50,079",'n',release
8,"2021-10-20 19:58:50,662",Key.shift,press
9,"2021-10-20 19:58:50,873",'J',press


In [58]:
path = 'input_data/'
user_list = files = [os.path.splitext(filename)[0] for filename in listdir(path)]
for i in range(0,len(user_list),5):
    all_filenames = []
    user_name = user_list[i].split('_')[0] 
    for j in range(1, 6):    
        all_filenames = np.append(all_filenames, [path+user_name+'_'+str(j)+'.csv'])
    
    df_merged = (pd.read_csv(f, sep=';', names=columns, encoding = "ISO-8859-1") for f in all_filenames)
    df_merged = pd.concat(df_merged, ignore_index=False)
    df_merged.to_csv('input_data_merged/'+user_name+'.csv', sep=';')
display(df_merged)

,Timestamp,Key,Action
0,"2021-11-03 08:44:53,007",Key.enter,release
1,"2021-11-03 08:44:57,927",Key.shift,press
2,"2021-11-03 08:44:58,035",'P',press
3,"2021-11-03 08:44:58,126",'P',release
4,"2021-11-03 08:44:58,180",Key.shift,release
...,...,...,...
981,"2021-11-03 08:55:42,870",'e',release
982,"2021-11-03 08:55:42,986",Key.shift,press
983,"2021-11-03 08:55:43,015",'.',press
984,"2021-11-03 08:55:43,124",'.',release


# Preprocessing

In the event that a key is pressed for a 'long time' (at least 2 consecutive row with the same 'Key' value and 'Action' = press), we only keep the first pressed
<br>We also create some columns for specific keys like Shift, Alt, Win, Ctrl

In [82]:
path_merged = "input_data_merged/"
def is_shifted_key(df, key_index):
    press = False
    release = False
    for i in range(key_index, df.index.start-1, -1):
        if df['Key'][i] == 'Key.shift' and df['Action'][i] == 'press':
            press = True
            for j in range(key_index, i, -1):
                if df['Key'][j] == 'Key.shift' and df['Action'][j] == 'release':
                    release = True
                    break
            break
    if press == True and release == False:
        return True
    else:
        return False

def preprocess_shift_holding(dataframe):
    for i in range(dataframe.index.start, dataframe.index.stop):
        current_key = dataframe['Key'][i]
        if dataframe['Action'][i] == 'press':
            press = True
            j = i + 1
            while press:
                if j < dataframe.index.stop and dataframe['Action'][j] == 'press' and current_key == dataframe['Key'][j]:
                    dataframe = dataframe.drop([j])
                    dataframe = dataframe.reset_index(drop=True)
                    print(current_key)
                else:
                    break

    dataframe_bis = pd.DataFrame(columns=columns_bis)
    for i in range(dataframe.index.start, dataframe.index.stop):
        if is_shifted_key(dataframe, i):
            tmp = pd.DataFrame([[dataframe['Timestamp'][i], dataframe['Key'][i], dataframe['Action'][i], 1]], columns = columns_bis)
            dataframe_bis = dataframe_bis.append(tmp)
        else:
            tmp = pd.DataFrame([[dataframe['Timestamp'][i], dataframe['Key'][i], dataframe['Action'][i], 0]], columns = columns_bis)
            dataframe_bis = dataframe_bis.append(tmp)
    
    dataframe_bis = dataframe_bis.reset_index(drop=True)
    dataframe = dataframe_bis
    display(dataframe) 



user_list = os.listdir(path_merged)
print(user_list)
for user in range(len(user_list)):
    dataframe = pd.read_csv(path_merged+user_list[user], sep=";", names=columns, encoding = "ISO-8859-1")
    dataframe = dataframe.reset_index(drop=True)
    dataframe = dataframe.drop([0])
    dataframe = dataframe.reset_index(drop=True)
    preprocess_shift_holding(dataframe)




     




['benjamin.csv']
Key.shift
Key.shift
Key.shift
Key.shift
Key.shift
Key.shift
Key.shift
Key.shift
Key.shift
Key.shift
Key.shift
Key.shift
Key.shift
Key.shift
Key.shift
Key.shift
Key.shift
Key.shift
Key.shift
Key.shift
Key.shift
Key.shift
Key.shift


KeyError: 4712

In [68]:
dataframe['Timestamp'] = pd.to_datetime(dataframe['Timestamp'], format='%Y-%m-%d %H:%M:%S,%f')

ValueError: time data 'Timestamp' does not match format '%Y-%m-%d %H:%M:%S,%f' (match)

## Details

**columns_nd** = *nd for normalize data* <br>
**Hold Duration (HD)** = *Duration between press and release of the same key* <br>
**Hold time in digraph (HTD)** = *Dwell time with the context of the preceding keystroke* <br/>
**Cumulative Dwell Time (CDT)** = *Holding time for two consecutive keys without flying time* <br>
**Digraph latency (DL)** = *Flight time between two keystrokes*





In [7]:
columns_nd = ['Key1', 'Key2', 'HD', 'HTD', 'CDT', 'DL', 'Shift']# 'Shift', 'Alt', 'Ctrl', 'Win']
df = pd.DataFrame(columns = columns_nd)

def search_for_press(df):
    for i in range(df.index.start, df.index.stop):
        if df['Action'][i] == 'press':
            return i

def search_for_release(df, press_index):
    for i in range (press_index, df.index.stop):
        if df['Key'][i] == df['Key'][press_index] and df['Action'][i] == 'release':
            return i
        
def search_for_following_key(df, key_index):
    for i in range(key_index, df.index.stop):
        if df['Key'][i] != df['Key'][key_index] and df['Action'][i] == 'press':
            return i
                    
while dataframe.empty == False:
    end = False
    press_index = search_for_press(dataframe)
    release_index = search_for_release(dataframe, press_index)
    if dataframe.index.stop-dataframe.index.start == 2:
        end = True
    if end == False:
        following_key_press_index = search_for_following_key(dataframe, press_index)
        following_key_release_index = search_for_release(dataframe, following_key_press_index)
        Key1 = dataframe['Key'][press_index]
        Key2 = dataframe['Key'][following_key_press_index]
        HD = dataframe['Timestamp'][release_index] - dataframe['Timestamp'][press_index]
        HD = HD.total_seconds()
        HTD = dataframe['Timestamp'][following_key_release_index] - dataframe['Timestamp'][following_key_press_index]
        HTD = HTD.total_seconds()
        display
        CDT = HD + HTD
        DL = dataframe['Timestamp'][following_key_press_index] - dataframe['Timestamp'][release_index]
        DL = DL.total_seconds()
        shift = dataframe['Shift'][press_index]
        dataframe = dataframe.drop([press_index, release_index])
        dataframe = dataframe.reset_index(drop=True)
        tmp = pd.DataFrame([[Key1, Key2, HD, HTD, CDT, DL, shift]], columns = columns_nd)
        df = df.append(tmp)
    else:
        Key1 = dataframe['Key'][press_index]
        HD = dataframe['Timestamp'][release_index] - dataframe['Timestamp'][press_index]
        HD = HD.total_seconds()
        HTD = None
        CDT = None
        DL = None
        shift = dataframe['Shift'][press_index]
        dataframe = dataframe.drop([press_index, release_index])
        tmp = pd.DataFrame([[Key1, None, HD, HTD, CDT, DL, shift]], columns = columns_nd)
        df = df.append(tmp)

        
df = df.reset_index(drop=True)
       
display(df)
    



,Key1,Key2,HD,HTD,CDT,DL,Shift
0,Key.shift,'B',0.519,0.121,0.64,-0.24,1
1,'B','o',0.121,0.13,0.251,0.718,1
2,'o','n',0.130,0.092,0.222,0.157,0
3,'n',Key.shift,0.092,0.424,0.516,0.583,0
4,Key.shift,'J',0.424,0.097,0.521,-0.213,1
5,'J','o',0.097,0.125,0.222,0.639,1
6,'o','u',0.125,0.064,0.189,0.144,0
7,'u','r',0.064,0.123,0.187,0.124,0
8,'r',None,0.123,None,None,None,0


# Scaled Manhattan Verifier

In [21]:
path = 'reference/'
user_list = files = [os.path.splitext(filename)[0] for filename in listdir(path)]
print(user_list)
def scaled_manhattan_verifier(vector):
    distance_to_users = np.array((0,2))
    for user in range(len(user_list)):
        user_ref = pd.read_csv(file_path, sep=";", names=columns, encoding = "ISO-8859-1")
        distance = 0
        index = 0
        for i in range(vector.shape[0]):
            for j in range(user_ref.shape[0]):
                if user_ref['Key1'][j] == vector['Key1'][i]:
                    distance += abs(vector['HD_mean'][i] - user_ref['HD_mean'][j])/user_ref['HD_std']
                    distance += abs(vector['DL_mean'][i] - user_ref['DL_mean'][j])/user_ref['DL_std']
                    distance_to_users = np.append(distance_to_users, [[index, distance]])
        index += 1
        distance = 0
    minim = 0
    res = 0
    for i in range(len(distance_to_users)):
        if distance_to_users[i][1] < minim:
            minim = distance_to_users[i][1]
            res = i
    print('Vous êtes '+user_list[res])
                    
            
            
            

['yo']
